# Packages 

In [25]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [26]:
ranker_train_data_dir = '../data/rank_train_data'
rank_model_version = 'rank_lgbm_v3'

In [27]:
debug = True

if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model_version = 'w2v_v3'
nic_model_version = 'nic'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100



# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = f'../data/sub_files/{rank_model_version}.parque'
eval_sub_file = f'../data/sub_files/eval_{rank_model_version}.parque'

In [28]:
! mkdir {rank_model_dir}

mkdir: cannot create directory ‘../model_training/rank_lgbm_v3’: File exists


# Read data

In [29]:

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))


In [30]:
def get_all_candidates(data_type, task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version
                      ,model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn):
    
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    if data_type == 'test':
        original_file_name = f"sessions_{data_type}_{task}.parquet"
    else:
        original_file_name = f"sessions_{data_type}.parquet"
    original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    
    # get w2v weight
    w2v_pl = w2v_pl.with_columns(
        pl.lit([list(range(w2v_topn, 0, -1))]).alias('w2v_weight')
    )
    get_w2v_weight = pl.element().rank()*0
    nic_pl = nic_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                # parallel=True
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    cols = ['session_id', 'next_item_prediction', 'w2v_weight']
    combined_pl = w2v_pl.select(cols).join(nic_pl.select(cols), how='left', on='session_id', suffix='_nic')

    # combined_pl = combined_pl.with_columns(
    #     pl.concat_list([pl.col('next_item_prediction'), pl.col('next_item_prediction_nic')]).alias('next_item_prediction'), 
    #     pl.concat_list([pl.col('w2v_weight'), pl.col('w2v_weight_nic')]).alias('w2v_weight'), 
    # ).select(
    #     pl.all().exclude(['next_item_prediction_nic', 'w2v_weight_nic'])
    # )
    explode_cols = ['next_item_prediction', 'w2v_weight']
    combined_pl = (
            pl.concat([w2v_pl.select(cols).explode(explode_cols), nic_pl.select(cols).explode(explode_cols)], how='vertical')
                .groupby(['session_id', 'next_item_prediction'])
                .agg(
                    pl.col('w2v_weight').max().alias('w2v_weight')
                )
                .groupby('session_id')
                .agg(
                    pl.all()
                )
    )
    combined_pl = (
        combined_pl.join(original_pl, how='left', on='session_id')
            # .with_columns(
            #     pl.col('prev_items')
            # )
    )
    return combined_pl


In [31]:
train_cg_pl = get_all_candidates(data_type='train')
eval_cg_pl = get_all_candidates(data_type='eval')
test_cg_pl = get_all_candidates(data_type='test')

In [32]:
# eval_cg_pl.head().collect()

In [9]:
# train_cg_pl.with_columns(
#     pl.col('next_item_prediction').arr.lengths().alias('length')
#     , pl.col('next_item_prediction').arr.unique().arr.lengths().alias('unique_length')
# ).select(pl.col('length').sub(pl.col('unique_length')).alias('diff')).collect().to_series().value_counts().sort('counts',
#                                                                                                                 descending=True)

In [10]:
# train_cg_pl.collect().estimated_size(unit='gb')

In [11]:
# original_pl.collect().estimated_size(unit='gb')

In [12]:




# data_type = 'train'
# w2v_file = os.path.join(candidate_path, 
#                        candidate_file_name.format(
#                 task=task
#             , data_type=data_type
#             , model_version=w2v_model
#             , model_for_eval=model_for_eval
#             , topn=w2v_topn
#                        ))
# nic_file = os.path.join(candidate_path,
#             candidate_file_name.format(
#                 task=task
#                 , data_type=data_type
#                 , model_version=nic_model
#                 , model_for_eval=model_for_eval
#                 , topn=nic_topn
#                        ))
# if data_type == 'test':
#     original_file_name = f"sessions_{data_type}_{task}.parquet"
# else:
#     original_file_name = f"sessions_{data_type}.parquet"
# original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

# # get w2v weight
# w2v_pl = w2v_pl.with_columns(
#     pl.lit([list(range(w2v_topn, 0, -1))]).alias('w2v_weight').cast(pl.List(pl.Int8))
# ).collect()


In [13]:
# get_w2v_weight = pl.element().rank()*0
# nic_pl = nic_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                             # parallel=True
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int8))
# ).collect()

In [14]:
# train_cg_pl.collect()

In [15]:
# eval_cg_pl.collect().estimated_size(unit='gb')

In [16]:
# train_cg_pl.collect()

In [17]:
# print(f"{train_cg_pl.collect().shape}; {eval_cg_pl.collect().shape}; {test_cg_pl.collect().shape}")

In [18]:
# train_cg_pl.collect().head()

In [19]:
# eval_cg_pl.collect()

In [20]:
# get_w2v_weight = pl.element().rank()*0

# train_pl.with_columns(
#     # pl.lit([list(range(100, 0, -1))]).alias('w2v_weight')
#     pl.col('next_item_prediction').arr.lengths().alias('length')
#     , pl.col('next_item_prediction').arr.eval(get_w2v_weight, parallel=True).alias('w2v_weight')
#     # , pl.lit([0 for ele in range(pl.col('length'))])

# ).head().collect()

In [21]:
# dir(pl.element())

In [22]:
# w2v_pl.head().collect()


In [23]:

# def get_all_candidates(data_type):
#     w2v_file = os.path.join(candidate_path, f'task1_{data_type}_w2v_top100.parquet')
#     nic_file = os.path.join(candidate_path, f'task1_{data_type}_nic_top100.parquet')
#     w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
#     nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

#     cols_to_keep = ['session_id', 'next_item_prediction', 'cg_source']

#     w2v_pl = w2v_pl.with_columns(pl.lit('word2vec').alias('cg_source')).select(cols_to_keep).explode('next_item_prediction')
#     nic_pl = nic_pl.with_columns(pl.lit('next_item_counter').alias('cg_source')).select(cols_to_keep).explode('next_item_prediction')
#     # length = w2v_pl.collect().shape[0]
#     # w2v_pl = w2v_pl.with_columns(
#     #     pl.Series(values=[list(range(100, 0, -1))]*length, name='w2v_weight')
#     # )
#     cg_pl = (
#             pl.concat([w2v_pl, nic_pl], how='vertical')
#                 .groupby(['session_id', 'next_item_prediction'])
#                 .agg(
#                     pl.col('cg_source')
#                 )
#                 .with_columns(
#                     pl.when(pl.col('cg_source').arr.contains(pl.lit('word2vec'))).then(1).otherwise(0).alias('whether_w2v')
#                     , pl.when(pl.col('cg_source').arr.contains(pl.lit('next_item_counter'))).then(1).otherwise(0).alias('whether_nic')
#                 )
#                 .select(
#                     pl.all().exclude('cg_source')
#                 )
#     )
#     return cg_pl

In [24]:
if eval_cg:
    eval_topn = 300

    col = f"recall@{eval_topn}"

    eval_final = (
            eval_cg_pl
            .lazy()
            .with_columns(
                pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
            )
            .with_columns(
                pl.concat_list([pl.col('next_item'), pl.col('next_item_prediction')]).alias('mrr')
            )
            ).with_columns(
                pl.col('next_item_prediction').arr.head(eval_topn).arr.contains(pl.col('next_item')).mean().alias(col)

            )
    final_res = eval_final.select(
            pl.count().alias('total_sessions')
            , pl.col(col).mean()

        ).collect()
    print(final_res)
    del final_res

NameError: name 'eval_cg' is not defined

In [ ]:
# eval_cg_pl.head().collect()

In [33]:
sampled_train_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train.parquet'), n_rows=SAMPLE_NUM)
eval_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'), n_rows=SAMPLE_NUM)
test_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'), n_rows=SAMPLE_NUM)

# Ranker 

## Config 

In [34]:
estimator = 1000
early_stop_rounds = 10
if debug:
    early_stop_rounds = 5
    
target = 'target'
feature_cols = [
    # 'whether_w2v'
    # , 'whether_nic'
    'next_item_weight'
    , 'locale'
    , 'w2v_weight'
    , 'last_item_similarity'
    , 'prev_item_similarity'	
    , 'prev_length'
]

categorical_feature=['locale']

## Train model

In [35]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    n_estimators=estimator, 
    importance_type='gain',
    eval_at=[3]
)

In [36]:
sampled_train_candidates = sampled_train_candidates.collect()
sampled_train_candidates.shape


(1000, 12)

In [37]:
sampled_eval_candidates = eval_candidates.collect()
sampled_eval_candidates.shape

(1000, 12)

In [38]:
print(sampled_train_candidates.select('target').mean())


shape: (1, 1)
┌────────┐
│ target │
│ ---    │
│ f64    │
╞════════╡
│ 0.077  │
└────────┘


In [39]:
print(sampled_eval_candidates.select('target').mean())

shape: (1, 1)
┌────────┐
│ target │
│ ---    │
│ f64    │
╞════════╡
│ 0.005  │
└────────┘


In [40]:
session_lengths_train = sampled_train_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()
session_lengths_eval = sampled_eval_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()

In [41]:
ranker = ranker.fit(
    X=sampled_train_candidates[feature_cols].to_pandas(),
    y=sampled_train_candidates[target].to_pandas(),
    group=session_lengths_train,
    eval_set=[(sampled_train_candidates[feature_cols].to_pandas(), sampled_train_candidates[target].to_pandas()),
           (sampled_eval_candidates[feature_cols].to_pandas(), sampled_eval_candidates[target].to_pandas())
             ],
    eval_group=[session_lengths_train,
                session_lengths_eval
               ]
    , categorical_feature=categorical_feature
    , callbacks=[early_stopping(early_stop_rounds)]
)

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:621: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['locale']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overridin

Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@3: 0.911078	valid_1's ndcg@3: 0.901461


In [42]:
print(sampled_train_candidates.estimated_size(unit='gb'))
del sampled_train_candidates

0.00016786716878414154


In [43]:
print(sampled_eval_candidates.estimated_size(unit='gb'))
del sampled_eval_candidates

0.00016793422400951385


## Save model 

In [44]:
rank_model_dir

'../model_training/rank_lgbm_v3'

In [45]:
if not debug:
    joblib.dump(
            value=ranker,
            filename=os.path.join(rank_model_dir, 'model.pkl')
    )

In [46]:
if not debug:
    del ranker

## Load Model 

In [47]:
if not debug:
    ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))
ranker

LGBMRanker(eval_at=[3], importance_type='gain', metric='ndcg',
           n_estimators=1000, objective='lambdarank')

In [48]:
# ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))


### Importance 

In [49]:
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
0,next_item_weight,1.659768e+02
3,last_item_similarity,1.268494e+01
5,prev_length,3.879150e+00
2,w2v_weight,1.421090e-14
4,prev_item_similarity,7.105430e-15
1,locale,0.000000e+00


# Eval ranker 

In [50]:
def rank_rec(candidate_pl, feature_cols, ranker, topn=100):

    candidate_pl = candidate_pl.collect()
    print(f"candidate pl shape: {candidate_pl.shape}")
    inference = ranker.predict(candidate_pl[feature_cols].to_pandas())
    test_result = (candidate_pl
         .lazy()
         .with_columns(
             pl.Series(name='predict', values=inference)
         )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
         .groupby(['session_id'])
         .agg(
             pl.col('next_item_prediction')
             , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
         ).with_columns(
             pl.col('next_item').arr.get(0).alias('next_item')
         )
    )
    return test_result


In [51]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates,
                         feature_cols=feature_cols
                         , ranker=ranker
                        # , topn=10
                         )

candidate pl shape: (1000, 12)


In [52]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
1000,0.259259,0.005,0.005


# Test inference 

In [53]:
test_candidates = test_candidates.collect()
test_candidates.shape

(1000, 10)

In [54]:
inference = ranker.predict(test_candidates[feature_cols].to_pandas())

In [55]:
# inference

In [56]:
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test_result.head().collect()

In [57]:
test_pl.schema

{'prev_items': Unknown, 'locale': Utf8, 'session_id': Int64}

In [58]:
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [59]:
predictions.shape

(1000, 2)

In [60]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
if not debug:
    predictions.to_parquet(sub_file, engine='pyarrow')

In [ ]:
# # You can submit with aicrowd-cli, or upload manually on the challenge page.
# !aicrowd submission create -c task-1-next-product-recommendation -f {sub_file}